# Generate STAC from a set of files

Applicaiton packages rely on stac files for stage-in and stage-out. While we handle this through the use of libraries, we often need the "initial" stac input tos tart a workflow or input to an applicaiton package based on a set of local files. This script helps us create that by pointing at a local directory and creating STAC items from them.

In [1]:
from unity_sds_client.resources.collection import Collection

# stage_out packages
from datetime import datetime, timezone
from unity_sds_client.resources.dataset import Dataset
from unity_sds_client.resources.data_file import DataFile

import netCDF4
import os


In [2]:
# Create a collection
# for this collection, the name doens't really matter because we're creating it form local files. If we were getting this from the Unity data catalog or CMR, this information would be supplied on your behalf.
out_collection  = Collection("EMIT_L1B_RAD")

In [3]:

# Create a Dataset for the collection
# A dataset is a collection of files relating to some measurement or period of measurements. this is also known as a "granule" in earthdata. 
# The start and end times should be read from the file metadata or the file name, but because we're sinply using this for staging local data 
# products, not searching, it's not required to have accurate information here. If we are creating data files we want to persist and search on, 
# it's important to add corret metadata to aid in search and filtering.

dataset = Dataset(
    name='EMIT_L1B_RAD_001_20231206T160939_2334011_006', 
    collection_id=out_collection.collection_id, 
    start_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(),
    end_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(),
    creation_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(),
)

In [4]:
# Add output file(s) to the dataset
# Here we are adding the 2 files that make up this dataaset. we give them a 'type' and an href (location). 
dataset.add_data_file(DataFile("radiance", "/unity/ads/inputs/L1B/EMIT_L1B_RAD_001_20231206T160939_2334011_006.nc"))
dataset.add_data_file(DataFile("observation", "/unity/ads/inputs/L1B/EMIT_L1B_OBS_001_20231206T160939_2334011_006.nc"))

In [5]:
# Add the dataset to the collection
#out_collection.add_dataset(dataset)
out_collection._datasets.append(dataset)

Collection.to_stac(out_collection, "/unity/ads/inputs/L1B")